# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 14 2022 12:26PM,250701,10,8775364,Yusen – 3D Matrix,Released
1,Nov 14 2022 12:23PM,250694,10,0086170413,ISDIN Corporation,Released
2,Nov 14 2022 12:23PM,250694,10,0086170409,ISDIN Corporation,Released
3,Nov 14 2022 12:23PM,250694,10,0086167489,ISDIN Corporation,Released
4,Nov 14 2022 12:23PM,250694,10,0086167490,ISDIN Corporation,Released
5,Nov 14 2022 12:23PM,250694,10,0086167492,ISDIN Corporation,Released
6,Nov 14 2022 12:23PM,250694,10,0086167491,ISDIN Corporation,Released
7,Nov 14 2022 12:23PM,250694,10,0086167495,ISDIN Corporation,Released
8,Nov 14 2022 12:23PM,250694,10,0086167494,ISDIN Corporation,Released
9,Nov 14 2022 12:23PM,250694,10,0086167481,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,250697,Executing,2
28,250697,Released,1
29,250698,Released,1
30,250699,Released,9
31,250701,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250696,1.0,NaN
250697,2.0,1.0
250698,NaN,1.0
250699,NaN,9.0
250701,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250634,0.0,2.0
250638,0.0,5.0
250659,0.0,6.0
250664,16.0,2.0
250666,3.0,65.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250634,0,2
250638,0,5
250659,0,6
250664,16,2
250666,3,65


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250634,0,2
1,250638,0,5
2,250659,0,6
3,250664,16,2
4,250666,3,65


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250634,,2
1,250638,,5
2,250659,,6
3,250664,16,2
4,250666,3,65


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 14 2022 12:26PM,250701,10,Yusen – 3D Matrix
1,Nov 14 2022 12:23PM,250694,10,ISDIN Corporation
60,Nov 14 2022 12:23PM,250693,10,ISDIN Corporation
121,Nov 14 2022 12:23PM,250699,12,Hush Hush
130,Nov 14 2022 12:20PM,250698,12,Hush Hush
131,Nov 14 2022 12:08PM,250697,19,"AdvaGen Pharma, Ltd"
134,Nov 14 2022 12:03PM,250696,12,Hush Hush
135,Nov 14 2022 12:01PM,250695,20,"Andelyn Biosciences, Inc."
136,Nov 14 2022 11:36AM,250685,10,"Senseonics, Incorporated"
137,Nov 14 2022 11:34AM,250682,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 14 2022 12:26PM,250701,10,Yusen – 3D Matrix,,1
1,Nov 14 2022 12:23PM,250694,10,ISDIN Corporation,,59
2,Nov 14 2022 12:23PM,250693,10,ISDIN Corporation,,61
3,Nov 14 2022 12:23PM,250699,12,Hush Hush,,9
4,Nov 14 2022 12:20PM,250698,12,Hush Hush,,1
5,Nov 14 2022 12:08PM,250697,19,"AdvaGen Pharma, Ltd",2,1
6,Nov 14 2022 12:03PM,250696,12,Hush Hush,1,
7,Nov 14 2022 12:01PM,250695,20,"Andelyn Biosciences, Inc.",,1
8,Nov 14 2022 11:36AM,250685,10,"Senseonics, Incorporated",,1
9,Nov 14 2022 11:34AM,250682,10,ISDIN Corporation,,64


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 12:26PM,250701,10,Yusen – 3D Matrix,1,
1,Nov 14 2022 12:23PM,250694,10,ISDIN Corporation,59,
2,Nov 14 2022 12:23PM,250693,10,ISDIN Corporation,61,
3,Nov 14 2022 12:23PM,250699,12,Hush Hush,9,
4,Nov 14 2022 12:20PM,250698,12,Hush Hush,1,
5,Nov 14 2022 12:08PM,250697,19,"AdvaGen Pharma, Ltd",1,2
6,Nov 14 2022 12:03PM,250696,12,Hush Hush,,1
7,Nov 14 2022 12:01PM,250695,20,"Andelyn Biosciences, Inc.",1,
8,Nov 14 2022 11:36AM,250685,10,"Senseonics, Incorporated",1,
9,Nov 14 2022 11:34AM,250682,10,ISDIN Corporation,64,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 12:26PM,250701,10,Yusen – 3D Matrix,1,
1,Nov 14 2022 12:23PM,250694,10,ISDIN Corporation,59,
2,Nov 14 2022 12:23PM,250693,10,ISDIN Corporation,61,
3,Nov 14 2022 12:23PM,250699,12,Hush Hush,9,
4,Nov 14 2022 12:20PM,250698,12,Hush Hush,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 12:26PM,250701,10,Yusen – 3D Matrix,1.0,NaN
1,Nov 14 2022 12:23PM,250694,10,ISDIN Corporation,59.0,NaN
2,Nov 14 2022 12:23PM,250693,10,ISDIN Corporation,61.0,NaN
3,Nov 14 2022 12:23PM,250699,12,Hush Hush,9.0,NaN
4,Nov 14 2022 12:20PM,250698,12,Hush Hush,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 12:26PM,250701,10,Yusen – 3D Matrix,1.0,0.0
1,Nov 14 2022 12:23PM,250694,10,ISDIN Corporation,59.0,0.0
2,Nov 14 2022 12:23PM,250693,10,ISDIN Corporation,61.0,0.0
3,Nov 14 2022 12:23PM,250699,12,Hush Hush,9.0,0.0
4,Nov 14 2022 12:20PM,250698,12,Hush Hush,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3509433,498.0,11.0
12,752093,10.0,1.0
15,752007,8.0,19.0
19,501366,5.0,2.0
20,250695,1.0,0.0
21,501347,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3509433,498.0,11.0
1,12,752093,10.0,1.0
2,15,752007,8.0,19.0
3,19,501366,5.0,2.0
4,20,250695,1.0,0.0
5,21,501347,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,498.0,11.0
1,12,10.0,1.0
2,15,8.0,19.0
3,19,5.0,2.0
4,20,1.0,0.0
5,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,498.0
1,12,Released,10.0
2,15,Released,8.0
3,19,Released,5.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Executing,11.0,1.0,19.0,2.0,0.0,0.0
Released,498.0,10.0,8.0,5.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Executing,11.0,1.0,19.0,2.0,0.0,0.0
1,Released,498.0,10.0,8.0,5.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Executing,11.0,1.0,19.0,2.0,0.0,0.0
1,Released,498.0,10.0,8.0,5.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()